# Грунина Маргарита. Вариант 4
# Составная формула Симпсона

$$u(x)+0.5\int_0^1 sh(xy) u(y)dy=x+0.5$$

In [1]:
from sympy import solve, series, factorial, oo, sinh, symbols
import scipy.integrate as integrate
from numpy.linalg import cond, solve
import matplotlib.pyplot as plt
import math

import pandas as pd

import numpy as np

In [2]:
N = 10
lambd = -0.5

# пределы интегрирования
a = 0
b = 1

f = lambda x: x + 0.5
delta = lambda x, y: 1 if x == y else 0

In [3]:
def dotL2(f1,f2):
    return integrate.quad(lambda x: f1(x) * f2(x), 0, 1)[0]

### Метод замены ядра на вырожденное

In [4]:
def singular_kernel(N):
    An = [lambda x, k=k: x**k / math.factorial(k) for k in range(1, 2 * N, 2)]
    Bn = [lambda x, k=k: x**k for k in range(1, 2 * N, 2)]

    A = np.zeros((N, N))
    b = np.zeros(N)

    for i in range(N):
        for j in range(N):
            A[i, j] = delta(i, j) - lambd * dotL2(Bn[i], An[j])

    for i in range(N):
        b[i] = dotL2(Bn[i], f)

    an = np.linalg.solve(A, b)

    u = lambda x: f(x) + lambd * sum([an[i] * An[i](x) for i in range(N)])

    x = np.linspace(0,1,1000)

    def loss(u):
        left_side = lambda x: u(x) - lambd * dotL2(lambda y: np.sinh(x * y), u)
        left_side_val = np.array([left_side(xi) for xi in x])
        return np.max(np.abs(left_side_val - f(x)))

    return u, loss(u)

In [13]:
data = {'N': [], 'Невязка': []}

for N in range(1,11):
    _, loss = singular_kernel(N)
    data['N'].append(N)
    data['Невязка'].append(loss)

df = pd.DataFrame(data)

In [12]:
df

,N,Невязка
0,1,2.372495e-02
1,2,7.987547e-04
2,3,1.444093e-05
3,4,1.618458e-07
4,5,1.233786e-09
5,6,6.811218e-12
6,7,2.864375e-14
7,8,4.440892e-16
8,9,2.220446e-16


### Метод механических квадратур

In [14]:
u_true, _ = singular_kernel(40)

In [15]:
def K(x, y):
    return np.sinh(x * y)

def mech_quad(a, b, N, lambd):
    x = np.linspace(a, b, N)
    h = (b - a) / (N - 1)
    A = np.eye(N)
    F = f(x)

    for n in range(N):
        for m in range(N):
            if m == 0 or m == N - 1:
                cm = h / 3
            elif m % 2 == 0:
                cm = 2 * h / 3
            else:
                cm = 4 * h / 3
            A[n, m] -= lambd * cm * K(x[n], x[m])

    U = solve(A, F)
    return U

def loss(u, x):
    return np.max(np.abs(u - u_true(x)))


In [16]:
Ns = [11, 21, 41, 81, 101, 201]


results = pd.DataFrame(columns=['N', 'Loss'])

for N in Ns:
    h = (b - a) / (N - 1)
    x = np.linspace(a, b, N)
    u = mech_quad(a, b, N, lambd)
    current_loss = loss(u, x)
    results = pd.concat([results, pd.DataFrame({'N': [N], 'Loss': [current_loss]})], ignore_index=True)

In [ ]:
results

,N,Loss
0,11,7.392747e-07
1,21,4.620707e-08
2,41,2.887962e-09
3,81,1.804981e-10
4,101,7.393242e-11
5,201,4.620748e-12
